In [1]:
import cv2
import os
import datetime
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.layers import Input, Activation, Conv2D, Flatten, Dense, MaxPooling2D
from keras.models import Model, load_model, Sequential
from keras.preprocessing.image import ImageDataGenerator#, img_to_array, array_to_img
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
from keras.utils.vis_utils import plot_model
from sklearn.model_selection import train_test_split
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input as mobilenet_preproc

tb_hist = TensorBoard(log_dir='./graph', histogram_freq=0, write_graph=True, write_images=True)

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:522: FutureWarn

# Load Dataset

In [2]:
import glob

In [3]:
imgs = glob.glob('./mrlEyes_2018_01/*/*.png')

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
train_imgs, test_imgs = train_test_split(imgs, test_size=0.15)

In [6]:
train_imgs, valid_imgs = train_test_split(train_imgs, test_size=0.18)

In [7]:
len(train_imgs), len(valid_imgs), len(test_imgs)

(59173, 12990, 12735)

In [8]:
from collections import Counter
from PIL import Image

In [9]:
x_train = np.empty((len(train_imgs), 224, 224, 3))
y_train = np.empty(len(train_imgs))

x_valid = np.empty((len(valid_imgs), 224, 224, 3))
y_valid = np.empty(len(valid_imgs))

x_test = np.empty((len(test_imgs), 224, 224, 3))
y_test = np.empty(len(test_imgs))

In [10]:
for idx, train in enumerate(train_imgs):
    x_train[idx] = np.expand_dims(np.array(Image.open(train).resize((224, 224), Image.BICUBIC)), -1)
    y_train[idx] = int(train.split('/')[-1].split('_')[4])

In [11]:
for idx, valid in enumerate(valid_imgs):
    x_valid[idx] = np.expand_dims(np.array(Image.open(valid).resize((224, 224), Image.BICUBIC)), -1)
    y_valid[idx] = int(valid.split('/')[-1].split('_')[4])

In [ ]:
for idx, test in enumerate(test_imgs):
    x_test[idx] = np.expand_dims(np.array(Image.open(test).resize((224, 224), Image.BICUBIC)), -1)
    y_test[idx] = int(test.split('/')[-1].split('_')[4])

In [ ]:
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape, x_test.shape, y_test.shape

# Preview

In [ ]:
plt.subplot(2, 1, 1)
plt.title(str(y_train[0]))
plt.imshow(x_train[0].reshape((224, 224)), cmap='gray')
plt.subplot(2, 1, 2)
plt.title(str(y_valid[4]))
plt.imshow(x_valid[4].reshape((224, 224)), cmap='gray')

In [ ]:
n_features = 512

def preprocess_input(x):
    img = x[:,:,::-1]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img


def mobilenet_model():
    base_model = MobileNet(weights='imagenet', include_top=True, 
                     input_shape=(224, 224, 3))
    base_model.layers.pop()
    x = base_model.layers[-1].output
    x = Dense(512)(x)
    x = Flatten()(x)
    #n_classes = Dense(6)(x)
    n_classes = Dense(1)(x)
    model = Model(input = base_model.input, output = n_classes)
    #opt = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None)
    opt = keras.optimizers.Adam()
    model.compile(
        #loss='categorical_crossentropy',
        loss='mean_squared_error',
        optimizer=opt,
        metrics=['accuracy'])
    print(model.summary())
    return model

# Data Augmentation

In [ ]:

shape_used = (224, 224)
epochs = 100
batch_size = 32
n_patience = 30

traingen = ImageDataGenerator(
    preprocessing_function = mobilenet_preproc,
    rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    #zoom_range=[0.8,1.2],
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    zca_whitening=False)
            
train_generator = traingen.flow(
    x=x_train, y=y_train,
    #target_size=shape_used,
    batch_size=batch_size,
    #class_mode="categorical",
    #class_mode='sparse',
    shuffle=True,
    seed=42)

validgen = ImageDataGenerator(
    preprocessing_function = mobilenet_preproc,
    rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    #zoom_range=[0.8,1.2],
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    zca_whitening=False)

valid_generator = validgen.flow(
    x=x_valid, y=y_valid,
    #target_size=shape_used,
    batch_size=batch_size,
    #class_mode="categorical",
    #class_mode='sparse',
    shuffle=True,
    seed=42)

testgen = ImageDataGenerator(
    preprocessing_function = mobilenet_preproc,
    rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    #zoom_range=[0.8,1.2],
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    zca_whitening=False)

test_generator = testgen.flow(
    x=x_test, y=y_test,
    #target_size=shape_used,
    batch_size=batch_size,
    #class_mode="categorical",
    #class_mode='sparse',
    shuffle=True,
    seed=42)

# Build Model and Train

In [ ]:
#model = custom_model()
model = mobilenet_model()


checkpoint = ModelCheckpoint('mobilenetEye.h5', monitor='val_loss',
                             verbose=1, save_best_only=True)
earlystop = EarlyStopping(monitor='val_loss', patience=n_patience, verbose=1)

hist = model.fit_generator(train_generator,
                    epochs=epochs,
                    validation_data=valid_generator,
                    workers=4,
                    callbacks=[earlystop,checkpoint,tb_hist])

In [ ]:
model.save('models/train_mobileEye_%s.h5' %(epochs))

5개의 layer를 가진 모델

In [ ]:
model.save('models/train_mrl_%s.h5' %(epochs_num))

In [ ]:
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
loss_ax.set_ylim([0.0, 0.5])

acc_ax.plot(hist.history['acc'], 'b', label='train acc')
acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')
acc_ax.set_ylim([0.8, 1.0])

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()

In [ ]:
scores = model.evaluate_generator(test_generator, steps=8)
print("## evaluation ##")
print(scores)
print("#################")
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

In [ ]:
scores = model.evaluate(x_test,y_test, batch_size=32)
print("## evaluation ##")
print(scores)
print("#################")
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

In [ ]:
scores = model.evaluate_generator(test_generator, steps=405)
print("## evaluation ##")
print(scores)
print("#################")
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

In [ ]:
scores = model.evaluate_generator(test_generator, steps=5)
print("## evaluation ##")
print(scores)
print("#################")
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

In [ ]:
scores = model.evaluate_generator(test_generator, steps=10)
print("## evaluation ##")
print(scores)
print("#################")
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

In [ ]:
#start_time = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
#
#model.fit_generator(
#    train_generator, epochs=50, validation_data=valid_generator,
#    callbacks=[
#        ModelCheckpoint('models/%s.h5' % (start_time), monitor='valid_acc', save_best_only=True, mode='max', verbose=1),
#        ReduceLROnPlateau(monitor='valid_acc', factor=0.2, patience=10, verbose=1, mode='auto', min_lr=1e-05)
#    ]
#)

In [ ]:
#model.save('models/train_mrl_0.h5')

In [ ]:
#model.save(‘train_mrl_0.hdf5’)

In [ ]:
#history['loss'], history['val_loss']

# Confusion Matrix

In [ ]:
#from sklearn.metrics import accuracy_score, confusion_matrix
#import seaborn as sns

#model = load_model('models/%s.h5' % (start_time))

#y_pred = model.predict(x_val/255.)
#y_pred_logical = (y_pred > 0.5).astype(np.int)

#print ('test acc: %s' % accuracy_score(y_val, y_pred_logical))
#cm = confusion_matrix(y_val, y_pred_logical)
#sns.heatmap(cm, annot=True)

# Distribution of Prediction

In [ ]:
#ax = sns.distplot(y_pred, kde=False)

---

# MobileNet V2

In [ ]:
#from keras.applications import MobileNetV2

In [ ]:
#model = MobileNetV2(input_shape=(32, 32, 1), include_top=True, weights=None, classes=2)

In [ ]:
#model.summary()